Sentence similarity with Sentence Transformers

In [1]:
import pandas as pd

In [2]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

/home/nmi/projects/dev/filter-discord-channels/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_csv('../data/vd-demojized.csv')

In [4]:
data.drop(['Author', 'Reactions'], axis=1, inplace=True)
data = data.map(lambda x: x.lower() if isinstance(x, str) else x)
print(len(data))
data[:5]

8822


,AuthorID,Content,Date
0,1174760528929902662,вітаю),2024-01-17t20:52:15.4270000+02:00
1,413028857147424778,:waving_hand:,2024-01-17t21:23:25.9710000+02:00
2,1174705952314896414,вітаю!,2024-01-17t21:34:04.6670000+02:00
3,1115913245363027988,pryvit!,2024-01-18t09:22:28.7310000+02:00
4,1084815314757365761,:saluting_face:,2024-01-18t09:56:20.0380000+02:00


In [5]:
# data = data[data['Content'].apply(len) > 50]
# print(len(data))
# data[:10]

In [6]:
import re

def filter_data(df):
    # Критерій 1: Виключення питань/відповідей, пов'язаних з лекціями, курсами, презентаціями
    keywords = ['тест', 'курс', 'лекція', 'презентація', 'посилання', 'відео', 'запис']
    mask_keywords = ~df['Content'].str.contains('|'.join(keywords), case=False, na=False)

    # Критерій 2: Видалення рядків із посиланнями
    url_pattern = r'(http[s]?://|www\.|\.com|\.ua|\.org|\.net)'
    mask_links = ~df['Content'].str.contains(url_pattern, na=False)

    # Поєднання критеріїв
    filtered_df = df[mask_keywords & mask_links]

    return filtered_df

# Фільтрація даних за першими двома критеріями
filtered_data = filter_data(data)

# Перегляд кількості залишених даних
print(filtered_data.shape)


(7015, 3)


/tmp/ipykernel_6287/3929482723.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_links = ~df['Content'].str.contains(url_pattern, na=False)


In [7]:
filtered_data[:20]

,AuthorID,Content,Date
0,1174760528929902662,вітаю),2024-01-17t20:52:15.4270000+02:00
1,413028857147424778,:waving_hand:,2024-01-17t21:23:25.9710000+02:00
2,1174705952314896414,вітаю!,2024-01-17t21:34:04.6670000+02:00
3,1115913245363027988,pryvit!,2024-01-18t09:22:28.7310000+02:00
4,1084815314757365761,:saluting_face:,2024-01-18t09:56:20.0380000+02:00
5,1134421507775397898,:waving_hand:,2024-01-18t10:27:42.3740000+02:00
6,674537666141749278,ура!,2024-01-18t12:28:28.3910000+02:00
8,1180949383731216408,чи можливо встановити vtx без куллера?,2024-01-18t13:46:59.8890000+02:00
9,1174705952314896414,можливо,2024-01-18t15:04:13.5530000+02:00
10,1197560822910701744,"доброго не вдається купити соло макс 2,5вт чи...",2024-01-18t17:57:02.5600000+02:00


In [8]:
filtered_data['ProcessedContent'] = filtered_data['Content'].apply(lambda text: model.encode(text))


/tmp/ipykernel_6287/422554855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['ProcessedContent'] = filtered_data['Content'].apply(lambda text: model.encode(text))


In [9]:
# sentences = [
#     filtered_data.iloc[25]['Content'],
#     filtered_data.iloc[26]['Content'],
#     filtered_data.iloc[27]['Content'],
#     filtered_data.iloc[28]['Content'],
#     ]

# sentences = [
#     'Хлопець ходить',
#     'Він одягнений в червону футболку',
#     'На хлопці червона футболка'
#     ]

# print(util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1]))
# print(util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[2]))
# print(util.pytorch_cos_sim(sentence_embeddings[1], sentence_embeddings[2]))

# print(sentences[0], '\n', sentences[1], '\n', sentences[2], '\n', sentences[3])


In [10]:
def calculate_similarity(e1, e2):
    return util.pytorch_cos_sim(e1, e2)

In [11]:
similarity_threshold = 0.7  # Adjust based on desired strictness
pairs = []
look_ahead = 20 # check next 20 messages for a similarities
max_i = len(filtered_data) - 1

for i, row in filtered_data.iterrows():
    c = 1
    max_sim = 0
    max_sim_i = c
    while c + i  < max_i and c < look_ahead:
        prev_sim = max_sim
        cur_sim = calculate_similarity(filtered_data.iloc[i]['ProcessedContent'], filtered_data.iloc[c]['ProcessedContent'])
        max_sim = max(prev_sim, cur_sim)
        if max_sim != prev_sim:
            max_sim_i = c + i

        c += 1

    if max_sim >= similarity_threshold:
        pairs.append({
            'Question': filtered_data.iloc[i]['Content'],
            'Answer': filtered_data.iloc[max_sim_i]['Content'],
            'Similarity': max_sim
        })

pairs_df = pd.DataFrame(pairs)


In [14]:
print(len(pairs_df))
pairs_df[20:30]

350


,Question,Answer,Similarity
20,дякую,"привіт, потребую поради по пайці. роблю вже тр...",[[tensor(1.)]]
21,це круто. дякую.,"у мене все працює, але наче інша побудова теми.",[[tensor(0.9036)]]
22,поки що ні,схоже закрили на доопрацювання,[[tensor(0.8310)]]
23,дякую!),просто плату польотного контролера можна підкл...,[[tensor(0.9770)]]
24,дякую,"ні, рама, стек та мотори мають бути тільки з р...",[[tensor(1.)]]
25,"так, буде",який припой оптимальний для збірки дронів? 60/...,[[tensor(0.7896)]]
26,так якраз 2-3мм. дякую,"вишліть фото і ціну у приватному повідомленні,...",[[tensor(0.7132)]]
27,дякую!,"якщо ця збірка, то так",[[tensor(0.9788)]]
28,дякую,"так ,вірно. але ,напевно мене збило інфо ,що e...",[[tensor(1.)]]
29,дякую!,"хм, ну так це ж ненадовго",[[tensor(0.9788)]]


In [13]:
pairs_df.to_csv("../data/pairs_test.csv", index=False)